In [2]:
!pip install ollama

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import json
import re
import ollama  # since you used this for Agent 3

# === Prompt template for Agent 5 ===
SYSTEM_PROMPT = """
You are Agent 5, responsible for repairing JSON-based frontend code.

## Input:
- Raw JSON code (from Agent 3 or previous repair attempt)
- Validation issues (from Agent 4)

## Task:
- Fix the JSON code so that it becomes valid and resolves the validation issues.
- Do NOT add explanations, comments, or extra text.
- Only output JSON in this structure:

For Example:
{
  "frontend": {
    "index.html": "...",
    "src/App.jsx": "...",
    "src/main.jsx": "...",
    "src/index.css": "...",
    "tailwind.config.js": "...",
    "vite.config.js": "...",
    "package.json": "...",
    "...other files..."
  }
}

## Rules:
- If a file is marked as "missing required file", you MUST generate it.
- If an import is marked as "broken", fix the path OR create the missing file.
- Always include `index.html` at project root.
- Always export default `App` in `src/App.jsx`.
- No markdown fences (```).
- No trailing commas.
- Preserve file contents (HTML, CSS, JS, JSX).
- Always return syntactically valid JSON.
- If unsure, attempt the best fix but ALWAYS return valid JSON.
- A simple rule the code you are going to generate should display the UI for that certain task..
"""

# === Utility: Clean JSON string ===
def clean_json_output(raw: str) -> str:
    """Remove fences, trailing commas, ensure clean JSON string."""
    if not raw:
        return "{}"
    cleaned = re.sub(r"```(?:json)?", "", raw, flags=re.IGNORECASE).strip()
    cleaned = cleaned.replace("```", "").strip()
    cleaned = re.sub(r",\s*([\]}])", r"\1", cleaned)
    return cleaned

# === Agent 5 function ===
def agent5_repair(code: dict, issues: dict, model: str = "deepseek-coder:6.7b") -> dict:
    """
    Uses Qwen LLM to repair JSON code.
    - code: dict from Agent 3 (or last repair)
    - issues: dict from Agent 4
    Returns: repaired JSON dict
    """
    user_message = f"""
Here is the current code JSON:
{json.dumps(code, indent=2)}

Here are the validation issues:
{json.dumps(issues, indent=2)}

Fix the code and return only JSON.
"""

    # Call Qwen
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ]
    )

    raw_output = response["message"]["content"]

    cleaned = clean_json_output(raw_output)
    try:
        parsed = json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("❌ Agent 5 produced invalid JSON")
        print(f"[DEBUG] Raw output:\n{raw_output}")
        return {}

    return parsed

# === Test Run (Standalone) ===
fake_agent3_output = {
  "src/App.jsx": "<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n  <meta charset=\"UTF-8\" />\n  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n  <title>Frontend App</title>\n  <link href=\"/index.css\" rel=\"stylesheet\" />\n</head>\n<body>\n  <div id=\"root\"></div>\n</body>\n</html>",
  "src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom/client';\nimport App from './App.jsx';\n\nconst root = ReactDOM.createRoot(document.getElementById('root'));\nroot.render(<App />);",
  "src/index.css": "@tailwind base;\n@tailwind components;\n@tailwind utilities;",
  "tailwind.config.js": "{\n  content: [\n    './src/**/*.{js,jsx,ts,tsx}',\n  ],\n  theme: {\n    extend: {},\n  },\n  plugins: [],\n}",
  "vite.config.js": "import { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\nexport default defineConfig({\n  plugins: [react()],\n});",
  "package.json": "{\n  \"name\": \"frontend-app\",\n  \"version\": \"1.0.0\",\n  \"private\": true,\n  \"scripts\": {\n    \"dev\": \"vite\",\n    \"build\": \"vite build\",\n    \"serve\": \"vite preview\"\n  },\n  \"dependencies\": {\n    \"react\": \"^18.2.0\",\n    \"react-dom\": \"^18.2.0\",\n    \"tailwindcss\": \"^3.0.0\",\n    \"@headlessui/react\": \"^1.4.0\",\n    \"@heroicons/react\": \"^1.0.6\"\n  },\n  \"devDependencies\": {\n    \"vite\": \"^2.8.0\",\n    \"@vitejs/plugin-react\": \"^1.3.0\",\n    \"autoprefixer\": \"^10.4.7\",\n    \"postcss\": \"^8.4.6\"\n  }\n}",
  "src/store/store.js": "import { configureStore } from '@reduxjs/toolkit';\n\nexport const store = configureStore({\n  reducer: {\n    // Add reducers here\n  },\n});",
  "src/screens/Home.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\n\nconst Home = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"home\" className=\"flex-1\">\n      {/* Home content */}\n    </section>\n  </div>\n);\n\nexport default Home;",
  "src/screens/About.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\n\nconst About = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"about\" className=\"flex-1\">\n      {/* About content */}\n    </section>\n  </div>\n);\n\nexport default About;",
  "src/screens/Projects.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\nimport ProjectCard from '../components/ProjectCard.jsx';\n\nconst Projects = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"projects\" className=\"flex-1\">\n      {/* Project cards */}\n      <ProjectCard title=\"Project 1\" description=\"Description of project 1\" />\n      <ProjectCard title=\"Project 2\" description=\"Description of project 2\" />\n    </section>\n  </div>\n);\n\nexport default Projects;",
  "src/screens/Skills.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\nimport SkillList from '../components/SkillList.jsx';\n\nconst Skills = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"skills\" className=\"flex-1\">\n      {/* Skill list */}\n      <SkillList skills={['React', 'JavaScript', 'Tailwind CSS']} />\n    </section>\n  </div>\n);\n\nexport default Skills;",
  "src/screens/Contact.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\nimport ContactForm from '../components/ContactForm.jsx';\n\nconst Contact = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"contact\" className=\"flex-1\">\n      {/* Contact form */}\n      <ContactForm />\n    </section>\n  </div>\n);\n\nexport default Contact;",
  "src/components/Navbar.jsx": "import React from 'react';\n\nconst Navbar = () => (\n  <nav className=\"bg-gray-800\">\n    <div className=\"max-w-7xl mx-auto px-4 sm:px-6 lg:px-8\">\n      <div className=\"flex items-center justify-between h-16\">\n        <div className=\"flex-shrink-0\">\n          <a href=\"#home\" className=\"text-white text-lg font-bold\">Frontend App</a>\n        </div>\n        <div className=\"hidden md:block\">\n          <div className=\"ml-4 flex items-center space-x-4\">\n            <a href=\"#about\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">About</a>\n            <a href=\"#projects\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">Projects</a>\n            <a href=\"#skills\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">Skills</a>\n            <a href=\"#contact\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">Contact</a>\n          </div>\n        </div>\n      </div>\n    </div>\n  </nav>\n);\n\nexport default Navbar;",
  "src/components/ProjectCard.jsx": "import React from 'react';\n\nconst ProjectCard = ({ title, description }) => (\n  <div className=\"bg-white shadow-md rounded-lg overflow-hidden\">\n    <div className=\"px-4 py-5 sm:px-6\">\n      <h3 className=\"text-lg leading-6 font-medium text-gray-900\">{title}</h3>\n      <p className=\"mt-1 max-w-2xl text-sm text-gray-500\">{description}</p>\n    </div>\n  </div>\n);\n\nexport default ProjectCard;",
  "src/components/ContactForm.jsx": "import React from 'react';\n\nconst ContactForm = () => (\n  <form className=\"bg-white shadow-md rounded-lg overflow-hidden p-4\">\n    <h3 className=\"text-lg leading-6 font-medium text-gray-900\">Get in Touch</h3>\n    <div className=\"mt-5 grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-2\">\n      <div className=\"sm:col-span-2\">\n        <label htmlFor=\"email\" className=\"block text-sm font-medium text-gray-700\">Email address</label>\n        <input type=\"email\" name=\"email\" id=\"email\" autoComplete=\"email\" required className=\"mt-1 block w-full border border-gray-300 rounded-md shadow-sm py-2 px-4 sm:text-sm\" />\n      </div>\n      <div className=\"sm:col-span-2\">\n        <label htmlFor=\"message\" className=\"block text-sm font-medium text-gray-700\">Message</label>\n        <textarea id=\"message\" name=\"message\" rows={3} required className=\"mt-1 block w-full border border-gray-300 rounded-md shadow-sm py-2 px-4 sm:text-sm\"></textarea>\n      </div>\n    </div>\n    <div className=\"mt-5\">\n      <button type=\"submit\" className=\"w-full bg-indigo-600 text-white py-2 px-4 border border-transparent rounded-md shadow-sm hover:bg-indigo-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-indigo-500\">Send</button>\n    </div>\n  </form>\n);\n\nexport default ContactForm;",
  "src/components/SkillList.jsx": "import React from 'react';\n\nconst SkillList = ({ skills }) => (\n  <ul className=\"list-disc pl-5\">\n    {skills.map((skill, index) => (\n      <li key={index} className=\"text-gray-700 text-sm\">{skill}</li>\n    ))}\n  </ul>\n);\n\nexport default SkillList;"
}

# fake_agent4_issues = {
#     "errors": [
#     "=== Issues ==="
# "package.json is invalid JSON",
# "index.html missing <script type='module' src='/src/main.jsx'>",
# "App.jsx missing default export",
# "Broken import: ./App.jsx in App.jsx",
# "Broken import: ../components/FormInputs in CreateTaskForm.jsx",
#     ]
# }

fake_agent4_issues = {
    "errors": [
    "=== Issues ==="
"index.html missing <script type='module' src='/src/main.jsx'>"
"App.jsx missing default export"
"Broken import: ./App.jsx in main.jsx"
"Broken import: ../components/Navbar.jsx in About.jsx"
"Broken import: ../components/Navbar.jsx in Contact.jsx"
"Broken import: ../components/ContactForm.jsx in Contact.jsx"
"Broken import: ../components/FormInputs in CreateTaskForm.jsx"
"Broken import: ../components/Navbar.jsx in Home.jsx"
"Broken import: ../components/Navbar.jsx in Projects.jsx"
"Broken import: ../components/ProjectCard.jsx in Projects.jsx"
"Broken import: ../components/Navbar.jsx in Skills.jsx"
"Broken import: ../components/SkillList.jsx in Skills.jsx"
    ]
}

fixed = agent5_repair(fake_agent3_output, fake_agent4_issues)
print("\n✅ Fixed JSON Output:\n", json.dumps(fixed, indent=2))

❌ Agent 5 produced invalid JSON
[DEBUG] Raw output:
{
  "frontend": {
    "index.html": "<!\nDOCTYPE html>\n<html lang=\"en\">\n<head>\n  <meta charset=\"UTF-8\" />\n  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n  <title>Frontend App</title>\n  <link href=\"/index.css\" rel=\"stylesheet\" ></head>\n<body>\n  <div id=\"root\"></div>\n  <script type='module' src='/src/main.jsx'></script>\n</body>\n</html>",
    "src/App.jsx": "import React from 'react';\nconst App = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"home\" className=\"flex-1\">{/* Home content */}</section>\n  </div>\n);\nexport default App;",
    "src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom/client';\nimport App from './App.jsx';\nconst root = ReactDOM.createRoot(document.getElementById('root'));\nroot.render(<App />);",
    "src/index.css": "@tailwind base;\n@tailwind components;\n@tailwind utilities;",
    "tailw

**Grok**

In [ ]:
import json
import re
import ollama

# === Prompt template for Agent 5 ===
def agent5_repair(code: dict, issues: dict, arch_text: str, model: str = "deepseek-coder:6.7b") -> dict:
    """
    Repairs JSON code while preserving structure based on arch_text.
    - code: dict from Agent 3 (or last repair)
    - issues: dict from Agent 4
    - arch_text: UI specs from Agent 2
    Returns: repaired JSON dict
    """
    # Extract issues list (handle your format)
    issue_list = issues.get("errors", []) if isinstance(issues, dict) else issues.get("issues_found", [])
    if not issue_list:
        return code  # No issues, return original

    SYSTEM_PROMPT = f"""
You are a Code Debugger. Output ONLY valid JSON: {{"frontend": {{"file/path": "full fixed code", ...}}}}. Fix ONLY listed issues. PRESERVE ALL original files/content/structure—copy unchanged files verbatim from original. Do NOT delete/add files unless issue requires it. Match UI from arch_text.

Arch Text (UI specs to guide fixes): {arch_text}

Original code (MUST base fixes on this; copy all keys unless directly fixed):
{json.dumps(code, indent=2)}

Issues to fix (address each exactly; ignore unrelated noise like CreateTaskForm if not in original):
{chr(10).join([f"- {iss}" for iss in issue_list if not iss.startswith("Broken import: ../components/FormInputs")])}

Rules:
- For missing: Add <script type='module' src='/src/main.jsx'> to index.html if missing.
- For broken imports: Fix paths (e.g., ensure Navbar.jsx exists/imports resolve) or create missing files with minimal valid content (e.g., empty JSX component with export default).
- For App.jsx: Ensure export default (add if missing, e.g., "export default function App() { return <div />; }").
- Preserve: All screens (Home/About/Projects/Skills/Contact), components (Navbar/ProjectCard/ContactForm/SkillList), store.js from original; keep hardcoded samples (e.g., projects array).
- JSON: Escape \\n in strings; double quotes; no trailing commas. Valid JSX in .jsx files.
- Example Fix (few-shot): If issue="Broken import: ../components/Navbar.jsx in Home.jsx", copy Navbar.jsx content, ensure import is './components/Navbar.jsx'.

Return same # of files as original (~15 for this UI).
"""

    user_message = "Fix and output JSON only."  # Keep short to avoid context overflow.

    # Call Ollama with max context and low creativity
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_message}
            ],
            options={"temperature": 0.1, "num_ctx": 16384}  # Max context, low randomness
        )
        raw_output = response["message"]["content"]
    except Exception as e:
        print(f"❌ Ollama error: {e}")
        return code  # Fallback if API fails

    cleaned = clean_json_output(raw_output)
    try:
        parsed = json.loads(cleaned)
        if "frontend" not in parsed:
            parsed = {"frontend": parsed}  # Wrap if missing outer key
    except json.JSONDecodeError as e:
        print(f"❌ Invalid JSON: {e}\nRaw: {raw_output}")
        return code  # Fallback to original on parse fail

    return parsed

# === Utility: Clean JSON string ===
def clean_json_output(raw: str) -> str:
    """Remove fences, trailing commas, ensure clean JSON string."""
    if not raw:
        return "{}"
    cleaned = re.sub(r"```(?:json)?", "", raw, flags=re.IGNORECASE).strip()
    cleaned = cleaned.replace("```", "").strip()
    cleaned = re.sub(r",\s*([\]}])", r"\1", cleaned)
    return cleaned

# === Test Run (Standalone) ===
arch_text = """
# Frontend Architecture Specification

## Tech Stack
- React (frontend framework)
- Tailwind CSS (styling library)
- Vite (build tool)
- Redux Toolkit (state management)
- localStorage/sessionStorage (client-side storage)

## Mandatory Files
- src/App.jsx
- src/main.jsx
- public/index.html
- src/index.css
- tailwind.config.js
- vite.config.js
- package.json

## Screens
- src/screens/Home.jsx
- src/screens/About.jsx
- src/screens/Projects.jsx
- src/screens/Skills.jsx
- src/screens/Contact.jsx

## Components
- src/components/Navbar.jsx
- src/components/ProjectCard.jsx
- src/components/ContactForm.jsx
- src/components/SkillList.jsx (for Skills screen)

## State Management
Redux store located in `src/store` with following slices:
- navigation (navbar state management)
- projects (project card expansion/collapse)

## Data Storage
None (localStorage/sessionStorage will be used if data persistence is needed)

## UI Features
- Responsive design using Tailwind CSS utilities
- Animations and transitions for smooth user experience
- Smooth scrolling navigation between screens
- Modern, lightweight design
"""

fake_agent3_output = {
    "src/App.jsx": "<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n  <meta charset=\"UTF-8\" />\n  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n  <title>Frontend App</title>\n  <link href=\"/index.css\" rel=\"stylesheet\" />\n</head>\n<body>\n  <div id=\"root\"></div>\n</body>\n</html>",
    "src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom/client';\nimport App from './App.jsx';\n\nconst root = ReactDOM.createRoot(document.getElementById('root'));\nroot.render(<App />);",
    "src/index.css": "@tailwind base;\n@tailwind components;\n@tailwind utilities;",
    "tailwind.config.js": "{\n  content: [\n    './src/**/*.{js,jsx,ts,tsx}',\n  ],\n  theme: {\n    extend: {},\n  },\n  plugins: [],\n}",
    "vite.config.js": "import { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\nexport default defineConfig({\n  plugins: [react()],\n});",
    "package.json": "{\n  \"name\": \"frontend-app\",\n  \"version\": \"1.0.0\",\n  \"private\": true,\n  \"scripts\": {\n    \"dev\": \"vite\",\n    \"build\": \"vite build\",\n    \"serve\": \"vite preview\"\n  },\n  \"dependencies\": {\n    \"react\": \"^18.2.0\",\n    \"react-dom\": \"^18.2.0\",\n    \"tailwindcss\": \"^3.0.0\",\n    \"@headlessui/react\": \"^1.4.0\",\n    \"@heroicons/react\": \"^1.0.6\"\n  },\n  \"devDependencies\": {\n    \"vite\": \"^2.8.0\",\n    \"@vitejs/plugin-react\": \"^1.3.0\",\n    \"autoprefixer\": \"^10.4.7\",\n    \"postcss\": \"^8.4.6\"\n  }\n}",
    "src/store/store.js": "import { configureStore } from '@reduxjs/toolkit';\n\nexport const store = configureStore({\n  reducer: {\n    // Add reducers here\n  },\n});",
    "src/screens/Home.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\n\nconst Home = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"home\" className=\"flex-1\">\n      {/* Home content */}\n    </section>\n  </div>\n);\n\nexport default Home;",
    "src/screens/About.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\n\nconst About = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"about\" className=\"flex-1\">\n      {/* About content */}\n    </section>\n  </div>\n);\n\nexport default About;",
    "src/screens/Projects.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\nimport ProjectCard from '../components/ProjectCard.jsx';\n\nconst Projects = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"projects\" className=\"flex-1\">\n      {/* Project cards */}\n      <ProjectCard title=\"Project 1\" description=\"Description of project 1\" />\n      <ProjectCard title=\"Project 2\" description=\"Description of project 2\" />\n    </section>\n  </div>\n);\n\nexport default Projects;",
    "src/screens/Skills.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\nimport SkillList from '../components/SkillList.jsx';\n\nconst Skills = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"skills\" className=\"flex-1\">\n      {/* Skill list */}\n      <SkillList skills={['React', 'JavaScript', 'Tailwind CSS']} />\n    </section>\n  </div>\n);\n\nexport default Skills;",
    "src/screens/Contact.jsx": "import React from 'react';\nimport Navbar from '../components/Navbar.jsx';\nimport ContactForm from '../components/ContactForm.jsx';\n\nconst Contact = () => (\n  <div className=\"flex flex-col min-h-screen\">\n    <Navbar />\n    <section id=\"contact\" className=\"flex-1\">\n      {/* Contact form */}\n      <ContactForm />\n    </section>\n  </div>\n);\n\nexport default Contact;",
    "src/components/Navbar.jsx": "import React from 'react';\n\nconst Navbar = () => (\n  <nav className=\"bg-gray-800\">\n    <div className=\"max-w-7xl mx-auto px-4 sm:px-6 lg:px-8\">\n      <div className=\"flex items-center justify-between h-16\">\n        <div className=\"flex-shrink-0\">\n          <a href=\"#home\" className=\"text-white text-lg font-bold\">Frontend App</a>\n        </div>\n        <div className=\"hidden md:block\">\n          <div className=\"ml-4 flex items-center space-x-4\">\n            <a href=\"#about\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">About</a>\n            <a href=\"#projects\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">Projects</a>\n            <a href=\"#skills\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">Skills</a>\n            <a href=\"#contact\" className=\"text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium\">Contact</a>\n          </div>\n        </div>\n      </div>\n    </div>\n  </nav>\n);\n\nexport default Navbar;",
    "src/components/ProjectCard.jsx": "import React from 'react';\n\nconst ProjectCard = ({ title, description }) => (\n  <div className=\"bg-white shadow-md rounded-lg overflow-hidden\">\n    <div className=\"px-4 py-5 sm:px-6\">\n      <h3 className=\"text-lg leading-6 font-medium text-gray-900\">{title}</h3>\n      <p className=\"mt-1 max-w-2xl text-sm text-gray-500\">{description}</p>\n    </div>\n  </div>\n);\n\nexport default ProjectCard;",
    "src/components/ContactForm.jsx": "import React from 'react';\n\nconst ContactForm = () => (\n  <form className=\"bg-white shadow-md rounded-lg overflow-hidden p-4\">\n    <h3 className=\"text-lg leading-6 font-medium text-gray-900\">Get in Touch</h3>\n    <div className=\"mt-5 grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-2\">\n      <div className=\"sm:col-span-2\">\n        <label htmlFor=\"email\" className=\"block text-sm font-medium text-gray-700\">Email address</label>\n        <input type=\"email\" name=\"email\" id=\"email\" autoComplete=\"email\" required className=\"mt-1 block w-full border border-gray-300 rounded-md shadow-sm py-2 px-4 sm:text-sm\" />\n      </div>\n      <div className=\"sm:col-span-2\">\n        <label htmlFor=\"message\" className=\"block text-sm font-medium text-gray-700\">Message</label>\n        <textarea id=\"message\" name=\"message\" rows={3} required className=\"mt-1 block w-full border border-gray-300 rounded-md shadow-sm py-2 px-4 sm:text-sm\"></textarea>\n      </div>\n    </div>\n    <div className=\"mt-5\">\n      <button type=\"submit\" className=\"w-full bg-indigo-600 text-white py-2 px-4 border border-transparent rounded-md shadow-sm hover:bg-indigo-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-indigo-500\">Send</button>\n    </div>\n  </form>\n);\n\nexport default ContactForm;",
    "src/components/SkillList.jsx": "import React from 'react';\n\nconst SkillList = ({ skills }) => (\n  <ul className=\"list-disc pl-5\">\n    {skills.map((skill, index) => (\n      <li key={index} className=\"text-gray-700 text-sm\">{skill}</li>\n    ))}\n  </ul>\n);\n\nexport default SkillList;"
}

fake_agent4_issues = {
    "errors": [
        "index.html missing <script type='module' src='/src/main.jsx'>",
        "App.jsx missing default export",
        "Broken import: ./App.jsx in main.jsx",
        "Broken import: ../components/Navbar.jsx in About.jsx",
        "Broken import: ../components/Navbar.jsx in Contact.jsx",
        "Broken import: ../components/ContactForm.jsx in Contact.jsx",
        "Broken import: ../components/FormInputs in CreateTaskForm.jsx",
        "Broken import: ../components/Navbar.jsx in Home.jsx",
        "Broken import: ../components/Navbar.jsx in Projects.jsx",
        "Broken import: ../components/ProjectCard.jsx in Projects.jsx",
        "Broken import: ../components/Navbar.jsx in Skills.jsx",
        "Broken import: ../components/SkillList.jsx in Skills.jsx"
    ]
}

# Run Agent5
fixed = agent5_repair(fake_agent3_output, fake_agent4_issues, arch_text)
print("\n✅ Fixed JSON Output:\n", json.dumps(fixed, indent=2))